# Efficient Implementation of ROC AUC Score

## Introduction

This post is a continuation of the [ROC and AUC Interpretation](https://maitbayev.github.io/posts/roc-auc). 
Please make sure that you understand that post before reading this one. 

In this post, we will implement an efficient ROC AUC Score in Python with $O(n\log n)$ runtime complexity.

[Subscribe](https://maitbayev.substack.com/subscribe) to get a notification about future posts. 

## Explanation

In [1]:
#| echo: false
#| output: false

import numpy as np
import pandas as pd
import plotly.graph_objects as go

LIGHT_RED = "#ff8886"
LIGHT_GREEN = "lightgreen"
DARK_RED = "#8b0000"

def make_dataframe(positives, negatives, threshold: float = 0):
    positives = np.array(positives)
    negatives = np.array(negatives)
    df_positives = pd.DataFrame({
        "score": positives,
        "target": np.ones(len(positives)),
        "color": np.full(len(positives), LIGHT_GREEN),
        "stroke_width": (positives > threshold) * 3,
        "stroke_color": np.full(len(positives), "green"),
    })
    df_negatives = pd.DataFrame({
        "score": negatives,
        "target": np.zeros(len(negatives)),
        "color": np.full(len(negatives), LIGHT_RED),
        "stroke_width": (negatives > threshold) * 3,
        "stroke_color": np.full(len(negatives), DARK_RED),
    })
    df = pd.concat([df_positives, df_negatives])
    df.sort_values("score", ascending=False, inplace=True)
    return df


def figure_auc1d(positives, negatives, threshold: float = 0.5, reverse=True):
    df = make_dataframe(positives, negatives, threshold)
    return go.Figure(data=[
        go.Scatter(
            x=df["score"],
            y=np.full(len(df), 0.5),
            mode="markers",
            marker=dict(
                size=df["score"] * 40,
                color=df["color"],
                opacity=1,
                line=dict(
                    width=df["stroke_width"],
                    color=df["stroke_color"]
                ),
            )
        ),
        go.Scatter(
            x=[threshold, threshold],
            y=[0, 1],
            mode="lines",
            line=dict(
                color="black",
                dash="dot",
            ),
            visible=(0 <= threshold <= 1)
        )
    ], layout=go.Layout(
        plot_bgcolor="#ffffff",
        height=100,
        margin=dict(l=5, r=5, t=20, b=20),
        xaxis=dict(
            linecolor="#cccccc",
            autorange="reversed" if reverse else True,
            nticks=10,
            range=[0, 1],
        ),
        yaxis=dict(visible=False),
        legend=dict(visible=False)
    ))

Let's say we have:

- A dataset with positive and negative items
- An ML model that predicts a probability score from 0 to 1, representing the probability that the input belongs to the positive class.

Similar to the previous post, we can visualize our dataset and their probability predictions in the same visualization as following:

In [2]:
#| echo: false
positives = [0.25, 0.35, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
negatives = [0.1, 0.15, 0.2, 0.3, 0.4, 0.6, 0.75, 0.95]

figure_auc1d(positives, negatives, threshold=1.1, reverse=False).show(renderer="iframe")

Positive items in the dataset are shown in green, and negative items are shown in red. The sizes of circles represent the predicted probability scores, with smaller circles representing scores close to 0 and larger circles representing scores close to 1.

Then, choosing a threshold gives us different points on the ROC curve with x representing the false positive rate (FPR) and y representing the true positive rate (TPR). For example:

In [3]:
#| echo: false
figure_auc1d(positives, negatives, threshold=0.5, reverse=False).show(renderer="iframe")

A threshold of value 0.5 gives us the point $(\frac{3}{8}, \frac{4}{7})$, whereas a threshold of value 0.4:



In [4]:
#| echo: false
figure_auc1d(positives, negatives, threshold=0.4, reverse=False).show(renderer="iframe")

gives us the point $(\frac{3}{8}, \frac{5}{7})$. We just increased the numerator of the true positive rate when transitioning from 0.5 to 0.4, since a new green point was introduced as a true positive item; everything else remains the same. We would increase the numerator of the false positive rate if a new red point was introduced instead.


Next, we need to calculate the sum of the areas of all trapezoids formed by the mentioned adjacent points:

![trapezoids](images/trapezoids.jpg)

We would find the sum of all five trapezoids shown above, which will be the ROC AUC score.

## Implementation

Let's setup our environment:

In [5]:
import numpy as np

np.random.seed(0)
n = 100
target = np.random.randint(0, 2, n)
predicted = np.random.rand(n)

We randomly generated targets and predicted probability scores. Let's check the result of `sklearn.metrics.roc_auc_score`:

In [6]:
import sklearn
sklearn.metrics.roc_auc_score(target, predicted)

np.float64(0.4277597402597403)

Our implementation should have the same score. 

### Trapezoid Area

First, let's implement a helper function that finds the area of the trapezoid defined by two points $(x_0, y_0)$ and $(x_1, y_1)$.

![Area of trapezoid](images/trapezoid.jpg)

To achieve this, we can add the area of the rectangle and the area of the right triangle, which is:

$$
\begin{align}
\text{Area}&=(x_1-x_0) \times y0+\frac{1}{2}(x_1-x_0) \times (y_1-y_0)\\
&= \frac{1}{2}(x_1-x_0) \times (2y_0+y_1 - y_0)\\
&= \frac{1}{2}(x_1-x_0) \times (y_0 + y_1)\\
\end{align}
$$

Let's implement the formula in Python:

In [7]:
def trapezoid_area(p0, p1):
    return (p1[0] - p0[0]) * (p0[1] + p1[1]) / 2.0

### ROC AUC Score 

It is better explained in the code, but roughtly our algorithm is:

1. Sort items by their predicted scores, from largest to smallest
2. Process the sorted items one by one in a loop
    1. Form the current point on the ROC curve by: $(\frac{\text{num\_false\_positive}}{\text{num\_negative}}, \frac{\text{num\_true\_positive}}{\text{num\_positive}})$
    2. Add the trapezoid area formed by the previous point and the current one 
    3. If the current item is positive, then increase `num_true_positive` by one
    4. If the current item is negative, then increase `num_false_positive` by one


In [8]:
def roc_auc_score(target, predicted):
    n = target.shape[0]
    num_positive = np.sum(target == 1)
    num_negative = n - num_positive 
    
    order = np.argsort(predicted)[::-1]
    last = [0, 0]
    num_true_positive = 0
    num_false_positive = 0
    score = 0
    for index in range(n):
        # Make sure that the new threshold is unique
        if index == 0 or predicted[order[index]] != predicted[order[index - 1]]:
            # True positive rate
            tpr = num_true_positive / num_positive
            # False positive rate
            fpr = num_false_positive / num_negative
            # New point on the ROC curve
            cur = [fpr, tpr]
            
            score += trapezoid_area(last, cur)
            last = cur
        
        if target[order[index]] == 1:
            num_true_positive += 1
        else:
            num_false_positive += 1
    score += trapezoid_area(last, [1, 1])

    return score 

In [9]:
roc_auc_score(target, predicted)

np.float64(0.4277597402597403)

Nice, we got exactly the same result as `sklearn` 

## The End

I hope you enjoyed this post.

[Subscribe](https://maitbayev.substack.com/subscribe) to get a notification about future posts.